<a href="https://colab.research.google.com/github/atstuyuki/Scapula3Danalysis/blob/main/%E7%AD%8B%E7%96%B2%E5%8A%B4MMG%E8%A7%A3%E6%9E%90_20260114.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### 処理全体の概要

WAVファイルをアップロードし、筋音図解析用に20-200Hzのフィルタ処理とダウンサンプリングを行います。その後、PSD、エンベロープ、MFCC、ZCR等の特徴量を抽出し、データ拡張を行った上で、結果をCSVとスペクトログラム画像（機械学習用・確認用）として保存します。

---

### CSVパラメータの意義

* **file**: 解析元のファイル名。データの出自（どの被験者や試行か）を特定するために使用します。
* **condition / trial**: 実験条件（pre/postなど）や試行回数のラベル。データを群分けして比較する際に使用します。
* **sr_raw / sr_proc**: 元データと解析用（処理後）のサンプリング周波数。時間解像度の確認や記録として残します。
* **bp_low / bp_high**: 適用したバンドパスフィルタの下限と上限。解析対象とした周波数帯域の定義情報です。
* **notch_peak_40_70**: 自動検知・除去された電源ノイズ（ハムノイズ）のピーク周波数。ノイズ混入の有無を確認します。
* **rms_20_200**: フィルタ処理後の信号の実効値。筋活動の全体的な強度（振幅の大きさ）を表す基本指標です。
* **zcr**: 信号がゼロをまたぐ頻度。信号の「細かさ」を表し、筋音図では高周波ノイズの混入具合等の目安になります。
* **env_mean / peak / auc**: 波形の包絡線（活動の概形）の平均値、最大値、総面積。筋活動の量や持続的な大きさを評価します。
* **mean_freq / median_freq**: パワースペクトルの平均および中央周波数。筋疲労に伴う「周波数の低下（徐波化）」の評価に重要です。
* **peak_freq**: パワーが最も強い周波数。その動作において支配的な振動成分（筋線維の振動数など）を特定します。
* **total_power**: 20-200Hz帯域全体の総エネルギー。RMSと同様に活動強度を示しますが、周波数領域での積分値です。
* **power_XX / ratio_XX**: 特定の帯域ごとのパワー絶対値とその全体に対する割合。周波数成分の偏りを詳細に分析します。
* **mfcc**: メル周波数ケプストラム係数。波形の「音色」や「質感」を数値化したもので、機械学習の分類によく寄与します。
* **spec_png/npy_aug**: 生成・保存されたスペクトログラム画像および数値データのファイルパス。後からデータを確認する際の参照用です。

In [9]:
# -*- coding: utf-8 -*-
"""筋疲労MMG解析_Colab版_500Hz対応.ipynb

主な機能:
- SOSバンドパスフィルタによる安定した前処理
- 40-70 Hzの自動ノッチフィルタ（ハムノイズ除去）
- 解析用にダウンサンプリング
- Welch法によるPSD特徴量 + エンベロープ (移動RMS) + MFCC (時間平均)
- スペクトログラム画像（0-500 Hz）の出力 + データ拡張による画像生成

★修正点 (2026/01/14):
- バンドパスフィルタ上限(BP_HIGH)を 200Hz -> 500Hz に変更
- 解析用サンプリングレート(TARGET_SR)を 1000Hz -> 2000Hz に変更 (Nyquist周波数確保のため)
- スペクトログラムの表示範囲を 500Hz まで拡大
- CSVの特徴量カラムに 200-500Hz 帯域を追加
"""

from __future__ import annotations
import os
from pathlib import Path
import numpy as np
import soundfile as sf
import pandas as pd
from scipy.signal import butter, sosfiltfilt, iirnotch, filtfilt, welch, resample_poly
import librosa
import matplotlib.pyplot as plt

# Google Colab用ファイルアップロード
from google.colab import files


# ====== 設定（再現性のため固定） ======
BP_LOW = 20.0     # バンドパスフィルタ下限
BP_HIGH = 500.0   # ★変更: バンドパスフィルタ上限を500Hzへ
TARGET_SR = 2000  # ★変更: 500Hz解析のためサンプリングレートを2000Hzへ倍増

# スペクトログラム設定
N_FFT = 1024
HOP = 256
DB_VMIN = -80
DB_VMAX = 0

# MFCC設定
N_MFCC = 13

# データ拡張設定（保存する画像枚数）
N_SPECS = 5
AUG_SEED = 123


def to_mono(x: np.ndarray) -> np.ndarray:
    """入力がステレオ(2ch)の場合、平均をとってモノラルにします。"""
    if x.ndim == 2:
        return x.mean(axis=1)
    return x


def bandpass_sos(x: np.ndarray, sr: int, low: float, high: float, order: int = 6) -> np.ndarray:
    """
    数値的に安定したSOS（Second-Order Sections）方式を用いたバンドパスフィルタ。
    """
    sos = butter(order, [low / (sr / 2), high / (sr / 2)], btype="band", output="sos")
    return sosfiltfilt(sos, x)


def auto_notch_40_70(x: np.ndarray, sr: int, q: float = 30) -> tuple[np.ndarray, float]:
    """
    40Hz～70Hzの範囲で最も強いピーク（ハムノイズ等）を検出し、
    その周波数に対してノッチフィルタを適用して除去します。
    """
    f, p = welch(x, fs=sr, nperseg=min(len(x), 32768))
    m = (f >= 40) & (f <= 70)
    if not np.any(m):
        return x, float("nan")

    peak = float(f[m][np.argmax(p[m])])
    w0 = peak / (sr / 2)  # 正規化周波数
    b, a = iirnotch(w0, Q=q)
    y = filtfilt(b, a, x)
    return y, peak


def downsample(x: np.ndarray, sr: int, target_sr: int) -> tuple[np.ndarray, int]:
    """
    ポリフェーズフィルタを用いて、指定したサンプリングレート(target_sr)にダウンサンプリングします。
    """
    if sr == target_sr:
        return x, sr
    g = np.gcd(int(sr), int(target_sr))
    up = int(target_sr // g)
    down = int(sr // g)
    y = resample_poly(x, up, down)
    return y.astype(np.float64), target_sr


def moving_rms(x: np.ndarray, win_samples: int) -> np.ndarray:
    """
    矩形窓を用いた移動二乗平均平方根（RMS）を計算し、エンベロープ（包絡線）を求めます。
    """
    win_samples = max(1, int(win_samples))
    win = np.ones(win_samples, dtype=np.float64) / win_samples
    return np.sqrt(np.convolve(x * x, win, mode="same"))


def welch_features(x: np.ndarray, sr: int, fmin: float, fmax: float, nperseg: int = 1024) -> dict:
    """
    Welch法を用いて指定帯域[fmin, fmax]内のPSD特徴量を算出します。
    """
    f, p = welch(x, fs=sr, nperseg=min(nperseg, len(x)))
    band = (f >= fmin) & (f <= fmax)
    fb, pb = f[band], p[band]
    pb = np.maximum(pb, 1e-20)

    # 統計量
    if len(pb) == 0:
        mean_f, median_f, peak_f, total_power = 0.0, 0.0, 0.0, 0.0
    else:
        mean_f = float((fb * pb).sum() / pb.sum())
        cdf = np.cumsum(pb) / pb.sum()
        median_f = float(fb[np.searchsorted(cdf, 0.5)])
        peak_f = float(fb[np.argmax(pb)])
        total_power = float(pb.sum())

    # 帯域ごとのパワー計算
    # 500Hzまで対応できるように区分を追加
    bands = [(20, 50), (50, 80), (80, 120), (120, 200)]
    if fmax > 200:
        bands.append((200, int(fmax))) # 200Hz以上があれば追加 (例: 200-500)

    band_powers = {}
    for lo, hi in bands:
        if hi > fmax: hi = int(fmax) # 上限クリップ
        if lo >= hi: continue

        mm = (f >= lo) & (f <= hi)
        band_powers[f"power_{lo}_{hi}"] = float(p[mm].sum())

    ratios = {k.replace("power", "ratio"): (v / total_power if total_power > 0 else 0) for k, v in band_powers.items()}

    return {
        "mean_freq": mean_f,
        "median_freq": median_f,
        "peak_freq": peak_f,
        f"total_power_{int(fmin)}_{int(fmax)}": total_power,
        **band_powers,
        **ratios,
    }


def mfcc_features(x: np.ndarray, sr: int) -> dict:
    """
    MFCC（メル周波数ケプストラム係数）を計算し、各係数の時間平均を返します。
    """
    y = x.astype(np.float32)

    mfcc = librosa.feature.mfcc(
        y=y,
        sr=sr,
        n_mfcc=N_MFCC,
        n_fft=N_FFT,
        hop_length=HOP,
        window="hann",
        center=True,
    )
    mfcc_mean = mfcc.mean(axis=1)
    return {f"mfcc{idx + 1:02d}_mean": float(val) for idx, val in enumerate(mfcc_mean)}


def shift_with_zeros(x: np.ndarray, shift: int) -> np.ndarray:
    """
    ゼロパディングを用いた時間シフトを行います。
    """
    n = len(x)
    if shift == 0:
        return x
    y = np.zeros_like(x)
    if shift > 0:
        if shift < n:
            y[shift:] = x[: n - shift]
    else:
        s = -shift
        if s < n:
            y[: n - s] = x[s:]
    return y


def save_spectrogram_png_and_npy(x: np.ndarray, sr: int, png_path: Path, npy_path: Path) -> None:
    """
    スペクトログラムを保存します。
    ★修正: BP_HIGH (500Hz) まで表示するように変更
    """
    S = librosa.stft(x, n_fft=N_FFT, hop_length=HOP, window="hann", center=True)
    P = np.abs(S) ** 2

    # 固定リファレンス (ref=1.0)
    P_db = librosa.power_to_db(P, ref=1.0)

    freqs = librosa.fft_frequencies(sr=sr, n_fft=N_FFT)

    # ★変更: BP_HIGH以下を抽出
    fmask = freqs <= BP_HIGH
    P_db_filtered = P_db[fmask, :]
    freqs_filtered = freqs[fmask]

    # フォルダ作成
    npy_path.parent.mkdir(parents=True, exist_ok=True)
    png_path.parent.mkdir(parents=True, exist_ok=True)

    # NPY保存
    np.save(npy_path, P_db_filtered.astype(np.float32))

    # PNG保存（軸なし）
    fig = plt.figure(figsize=(4, 4), dpi=200)
    ax = fig.add_axes([0, 0, 1, 1])  # マージンなし
    ax.imshow(
        P_db_filtered,
        aspect="auto",
        origin="lower",
        vmin=DB_VMIN,
        vmax=DB_VMAX,
        extent=[0, P_db_filtered.shape[1], freqs_filtered[0], freqs_filtered[-1]],
    )
    ax.set_axis_off()
    fig.savefig(png_path, bbox_inches="tight", pad_inches=0)
    plt.close(fig)


def make_augmented_signals(x: np.ndarray, sr: int, n: int = N_SPECS, seed: int = AUG_SEED) -> list[np.ndarray]:
    """
    スペクトログラム増強用の信号をn個生成します。
    ★修正: ノイズフィルタも BP_HIGH まで対応
    """
    rng = np.random.default_rng(seed)
    xs = [x]

    if n <= 1:
        return xs

    # 帯域制限ノイズ用のフィルタを作成 (BP_LOW ～ BP_HIGH)
    noise_sos = butter(4, [BP_LOW / (sr / 2), BP_HIGH / (sr / 2)], btype="band", output="sos")

    for _ in range(n - 1):
        y = x.copy()

        # 1) ゲイン変更
        gain = rng.uniform(0.80, 1.20)
        y *= gain

        # 2) 時間シフト
        shift_ms = rng.uniform(-50, 50)
        shift = int((shift_ms / 1000.0) * sr)
        y = shift_with_zeros(y, shift)

        # 3) 帯域制限ノイズ付加
        sig_rms = float(np.sqrt(np.mean(y ** 2)) + 1e-12)
        snr_db = float(rng.uniform(25, 40))
        noise_rms = sig_rms / (10 ** (snr_db / 20))

        wn = rng.normal(0.0, 1.0, size=y.shape).astype(np.float64)
        bn = sosfiltfilt(noise_sos, wn)
        bn = bn / (np.sqrt(np.mean(bn ** 2)) + 1e-12)
        y = y + bn * noise_rms

        y = np.clip(y, -1.0, 1.0)
        xs.append(y)

    return xs


def process_one(wav_path: Path, condition: str, trial_id: int, outdir: Path) -> dict:
    # 読み込み
    x, sr = sf.read(wav_path)
    x = to_mono(x).astype(np.float64)
    x = x - x.mean()

    # 前処理 (BP_LOW ～ BP_HIGH)
    x_bp = bandpass_sos(x, sr, BP_LOW, BP_HIGH, order=6)
    x_nt, notch_peak = auto_notch_40_70(x_bp, sr, q=30)

    # ダウンサンプリング
    x_ds, sr2 = downsample(x_nt, sr, TARGET_SR)

    # エンベロープ計算
    win = int(0.1 * sr2)
    env = moving_rms(x_ds, win_samples=max(3, win))
    env_mean = float(env.mean())
    env_peak = float(env.max())
    env_auc = float(env.sum() / sr2)

    # PSD特徴量 + 振幅特徴量 (fmax=BP_HIGH)
    feats = welch_features(x_ds, sr2, fmin=BP_LOW, fmax=BP_HIGH, nperseg=1024)
    rms = float(np.sqrt(np.mean(x_ds ** 2)))

    # MFCC特徴量
    mfcc_dict = mfcc_features(x_ds, sr2)

    # スペクトログラム生成
    aug_signals = make_augmented_signals(x_ds, sr2, n=N_SPECS, seed=AUG_SEED)
    spec_png_paths, spec_npy_paths = [], []

    for k, sig in enumerate(aug_signals, 1):
        png_path = outdir / "spec_png" / f"{condition}_trial{trial_id:02d}_aug{k:02d}.png"
        npy_path = outdir / "spec_npy" / f"{condition}_trial{trial_id:02d}_aug{k:02d}.npy"
        save_spectrogram_png_and_npy(sig, sr2, png_path, npy_path)
        spec_png_paths.append(str(png_path))
        spec_npy_paths.append(str(npy_path))

    # CSV用の行データを作成
    row = {
        "file": wav_path.name,
        "condition": condition,
        "trial": int(trial_id),
        "sr_raw": int(sr),
        "sr_proc": int(sr2),
        "bp_low": float(BP_LOW),
        "bp_high": float(BP_HIGH),
        "notch_peak_40_70": float(notch_peak),
        f"rms_{int(BP_LOW)}_{int(BP_HIGH)}": float(rms), # カラム名を動的に変更
        "env_mean": float(env_mean),
        "env_peak": float(env_peak),
        "env_auc": float(env_auc),
        **feats,
        **mfcc_dict,
        **{f"spec_png_aug{k:02d}": p for k, p in enumerate(spec_png_paths, 1)},
        **{f"spec_npy_aug{k:02d}": p for k, p in enumerate(spec_npy_paths, 1)},
    }
    return row




In [1]:
# -*- coding: utf-8 -*-
"""筋疲労MMG解析_Colab版_500Hz_Axis対応.ipynb

主な機能:
- SOSバンドパスフィルタによる安定した前処理
- 40-70 Hzの自動ノッチフィルタ（ハムノイズ除去）
- 解析用にダウンサンプリング (2000Hz)
- Welch法によるPSD特徴量 + エンベロープ (移動RMS) + MFCC (時間平均)
- スペクトログラム画像（0-500 Hz）の出力 (軸なし＆軸ありの両方を保存)
- データ拡張

★修正点:
- バンドパスフィルタ上限(BP_HIGH)を 500Hz に設定
- スペクトログラム保存時に「軸なし(ML用)」と「軸あり(確認用)」の2種類を出力するように変更
"""

from __future__ import annotations
import os
from pathlib import Path
import numpy as np
import soundfile as sf
import pandas as pd
from scipy.signal import butter, sosfiltfilt, iirnotch, filtfilt, welch, resample_poly
import librosa
import matplotlib.pyplot as plt

# Google Colab用ファイルアップロード
from google.colab import files


# ====== 設定（再現性のため固定） ======
BP_LOW = 20.0     # バンドパスフィルタ下限
BP_HIGH = 500.0   # バンドパスフィルタ上限 (500Hz)
TARGET_SR = 2000  # 解析用サンプリングレート (500Hz解析のため2000Hz)

# スペクトログラム設定
N_FFT = 1024
HOP = 256
DB_VMIN = -80
DB_VMAX = 0

# MFCC設定
N_MFCC = 13

# データ拡張設定（保存する画像枚数）
N_SPECS = 5
AUG_SEED = 123


def to_mono(x: np.ndarray) -> np.ndarray:
    """入力がステレオ(2ch)の場合、平均をとってモノラルにします。"""
    if x.ndim == 2:
        return x.mean(axis=1)
    return x


def bandpass_sos(x: np.ndarray, sr: int, low: float, high: float, order: int = 6) -> np.ndarray:
    """
    数値的に安定したSOS（Second-Order Sections）方式を用いたバンドパスフィルタ。
    """
    sos = butter(order, [low / (sr / 2), high / (sr / 2)], btype="band", output="sos")
    return sosfiltfilt(sos, x)


def auto_notch_40_70(x: np.ndarray, sr: int, q: float = 30) -> tuple[np.ndarray, float]:
    """
    40Hz～70Hzの範囲で最も強いピーク（ハムノイズ等）を検出し、
    その周波数に対してノッチフィルタを適用して除去します。
    """
    f, p = welch(x, fs=sr, nperseg=min(len(x), 32768))
    m = (f >= 40) & (f <= 70)
    if not np.any(m):
        return x, float("nan")

    peak = float(f[m][np.argmax(p[m])])
    w0 = peak / (sr / 2)  # 正規化周波数
    b, a = iirnotch(w0, Q=q)
    y = filtfilt(b, a, x)
    return y, peak


def downsample(x: np.ndarray, sr: int, target_sr: int) -> tuple[np.ndarray, int]:
    """
    ポリフェーズフィルタを用いて、指定したサンプリングレート(target_sr)にダウンサンプリングします。
    """
    if sr == target_sr:
        return x, sr
    g = np.gcd(int(sr), int(target_sr))
    up = int(target_sr // g)
    down = int(sr // g)
    y = resample_poly(x, up, down)
    return y.astype(np.float64), target_sr


def moving_rms(x: np.ndarray, win_samples: int) -> np.ndarray:
    """
    矩形窓を用いた移動二乗平均平方根（RMS）を計算し、エンベロープ（包絡線）を求めます。
    """
    win_samples = max(1, int(win_samples))
    win = np.ones(win_samples, dtype=np.float64) / win_samples
    return np.sqrt(np.convolve(x * x, win, mode="same"))


def welch_features(x: np.ndarray, sr: int, fmin: float, fmax: float, nperseg: int = 1024) -> dict:
    """
    Welch法を用いて指定帯域[fmin, fmax]内のPSD特徴量を算出します。
    """
    f, p = welch(x, fs=sr, nperseg=min(nperseg, len(x)))
    band = (f >= fmin) & (f <= fmax)
    fb, pb = f[band], p[band]
    pb = np.maximum(pb, 1e-20)

    # 統計量
    if len(pb) == 0:
        mean_f, median_f, peak_f, total_power = 0.0, 0.0, 0.0, 0.0
    else:
        mean_f = float((fb * pb).sum() / pb.sum())
        cdf = np.cumsum(pb) / pb.sum()
        median_f = float(fb[np.searchsorted(cdf, 0.5)])
        peak_f = float(fb[np.argmax(pb)])
        total_power = float(pb.sum())

    # 帯域ごとのパワー計算
    bands = [(20, 50), (50, 80), (80, 120), (120, 200)]
    if fmax > 200:
        bands.append((200, int(fmax))) # 200Hz以上があれば追加

    band_powers = {}
    for lo, hi in bands:
        if hi > fmax: hi = int(fmax) # 上限クリップ
        if lo >= hi: continue

        mm = (f >= lo) & (f <= hi)
        band_powers[f"power_{lo}_{hi}"] = float(p[mm].sum())

    ratios = {k.replace("power", "ratio"): (v / total_power if total_power > 0 else 0) for k, v in band_powers.items()}

    return {
        "mean_freq": mean_f,
        "median_freq": median_f,
        "peak_freq": peak_f,
        f"total_power_{int(fmin)}_{int(fmax)}": total_power,
        **band_powers,
        **ratios,
    }


def mfcc_features(x: np.ndarray, sr: int) -> dict:
    """
    MFCC（メル周波数ケプストラム係数）を計算し、各係数の時間平均を返します。
    """
    y = x.astype(np.float32)

    mfcc = librosa.feature.mfcc(
        y=y,
        sr=sr,
        n_mfcc=N_MFCC,
        n_fft=N_FFT,
        hop_length=HOP,
        window="hann",
        center=True,
    )
    mfcc_mean = mfcc.mean(axis=1)
    return {f"mfcc{idx + 1:02d}_mean": float(val) for idx, val in enumerate(mfcc_mean)}


def shift_with_zeros(x: np.ndarray, shift: int) -> np.ndarray:
    """
    ゼロパディングを用いた時間シフトを行います。
    """
    n = len(x)
    if shift == 0:
        return x
    y = np.zeros_like(x)
    if shift > 0:
        if shift < n:
            y[shift:] = x[: n - shift]
    else:
        s = -shift
        if s < n:
            y[: n - s] = x[s:]
    return y


def save_spectrogram_png_and_npy(x: np.ndarray, sr: int, png_path: Path, npy_path: Path) -> None:
    """
    スペクトログラムを保存します。
    ★修正: 「軸なし(ML用)」と「軸あり(確認用)」の2種類を保存
    """
    S = librosa.stft(x, n_fft=N_FFT, hop_length=HOP, window="hann", center=True)
    P = np.abs(S) ** 2

    # 固定リファレンス (ref=1.0)
    P_db = librosa.power_to_db(P, ref=1.0)

    freqs = librosa.fft_frequencies(sr=sr, n_fft=N_FFT)

    # BP_HIGH (500Hz) 以下を抽出
    fmask = freqs <= BP_HIGH
    P_db_filtered = P_db[fmask, :]
    freqs_filtered = freqs[fmask]

    # 時間軸（秒）の計算（軸あり表示用）
    duration = len(x) / sr

    # フォルダ作成
    npy_path.parent.mkdir(parents=True, exist_ok=True)
    png_path.parent.mkdir(parents=True, exist_ok=True)

    # --- 1. NPY保存 ---
    np.save(npy_path, P_db_filtered.astype(np.float32))

    # --- 2. PNG保存（軸なし: 機械学習用） ---
    fig = plt.figure(figsize=(4, 4), dpi=200)
    ax = fig.add_axes([0, 0, 1, 1])  # マージンなし
    ax.imshow(
        P_db_filtered,
        aspect="auto",
        origin="lower",
        vmin=DB_VMIN,
        vmax=DB_VMAX,
        extent=[0, P_db_filtered.shape[1], freqs_filtered[0], freqs_filtered[-1]],
    )
    ax.set_axis_off()
    fig.savefig(png_path, bbox_inches="tight", pad_inches=0)
    plt.close(fig)

    # --- 3. PNG保存（軸あり: 人間確認用） ---
    # ファイル名: 元の名前_axis.png
    png_path_axis = png_path.parent / f"{png_path.stem}_axis.png"

    fig2, ax2 = plt.subplots(figsize=(5, 4), dpi=200)
    img = ax2.imshow(
        P_db_filtered,
        aspect="auto",
        origin="lower",
        vmin=DB_VMIN,
        vmax=DB_VMAX,
        extent=[0, duration, freqs_filtered[0], freqs_filtered[-1]], # X軸を時間(秒)表示
        cmap='viridis'
    )
    ax2.set_title(f"Spectrogram (0-{int(BP_HIGH)}Hz)")
    ax2.set_xlabel("Time [sec]")
    ax2.set_ylabel("Frequency [Hz]")
    fig2.colorbar(img, ax=ax2, format='%+2.0f dB')

    fig2.tight_layout()
    fig2.savefig(png_path_axis)
    plt.close(fig2)


def make_augmented_signals(x: np.ndarray, sr: int, n: int = N_SPECS, seed: int = AUG_SEED) -> list[np.ndarray]:
    """
    スペクトログラム増強用の信号をn個生成します。
    """
    rng = np.random.default_rng(seed)
    xs = [x]

    if n <= 1:
        return xs

    # 帯域制限ノイズ用のフィルタ (BP_LOW ～ BP_HIGH)
    noise_sos = butter(4, [BP_LOW / (sr / 2), BP_HIGH / (sr / 2)], btype="band", output="sos")

    for _ in range(n - 1):
        y = x.copy()

        # 1) ゲイン変更
        gain = rng.uniform(0.80, 1.20)
        y *= gain

        # 2) 時間シフト
        shift_ms = rng.uniform(-50, 50)
        shift = int((shift_ms / 1000.0) * sr)
        y = shift_with_zeros(y, shift)

        # 3) 帯域制限ノイズ付加
        sig_rms = float(np.sqrt(np.mean(y ** 2)) + 1e-12)
        snr_db = float(rng.uniform(25, 40))
        noise_rms = sig_rms / (10 ** (snr_db / 20))

        wn = rng.normal(0.0, 1.0, size=y.shape).astype(np.float64)
        bn = sosfiltfilt(noise_sos, wn)
        bn = bn / (np.sqrt(np.mean(bn ** 2)) + 1e-12)
        y = y + bn * noise_rms

        y = np.clip(y, -1.0, 1.0)
        xs.append(y)

    return xs


def process_one(wav_path: Path, condition: str, trial_id: int, outdir: Path) -> dict:
    # 読み込み
    x, sr = sf.read(wav_path)
    x = to_mono(x).astype(np.float64)
    x = x - x.mean()

    # 前処理 (BP_LOW ～ BP_HIGH)
    x_bp = bandpass_sos(x, sr, BP_LOW, BP_HIGH, order=6)
    x_nt, notch_peak = auto_notch_40_70(x_bp, sr, q=30)

    # ダウンサンプリング (TARGET_SR=2000Hz)
    x_ds, sr2 = downsample(x_nt, sr, TARGET_SR)

    # エンベロープ計算
    win = int(0.1 * sr2)
    env = moving_rms(x_ds, win_samples=max(3, win))
    env_mean = float(env.mean())
    env_peak = float(env.max())
    env_auc = float(env.sum() / sr2)

    # PSD特徴量 + 振幅特徴量 (fmax=BP_HIGH)
    feats = welch_features(x_ds, sr2, fmin=BP_LOW, fmax=BP_HIGH, nperseg=1024)
    rms = float(np.sqrt(np.mean(x_ds ** 2)))

    # MFCC特徴量
    mfcc_dict = mfcc_features(x_ds, sr2)

    # スペクトログラム生成
    aug_signals = make_augmented_signals(x_ds, sr2, n=N_SPECS, seed=AUG_SEED)
    spec_png_paths, spec_npy_paths = [], []

    for k, sig in enumerate(aug_signals, 1):
        png_path = outdir / "spec_png" / f"{condition}_trial{trial_id:02d}_aug{k:02d}.png"
        npy_path = outdir / "spec_npy" / f"{condition}_trial{trial_id:02d}_aug{k:02d}.npy"
        save_spectrogram_png_and_npy(sig, sr2, png_path, npy_path)
        spec_png_paths.append(str(png_path))
        spec_npy_paths.append(str(npy_path))

    # CSV用の行データを作成
    row = {
        "file": wav_path.name,
        "condition": condition,
        "trial": int(trial_id),
        "sr_raw": int(sr),
        "sr_proc": int(sr2),
        "bp_low": float(BP_LOW),
        "bp_high": float(BP_HIGH),
        "notch_peak_40_70": float(notch_peak),
        f"rms_{int(BP_LOW)}_{int(BP_HIGH)}": float(rms),
        "env_mean": float(env_mean),
        "env_peak": float(env_peak),
        "env_auc": float(env_auc),
        **feats,
        **mfcc_dict,
        **{f"spec_png_aug{k:02d}": p for k, p in enumerate(spec_png_paths, 1)},
        **{f"spec_npy_aug{k:02d}": p for k, p in enumerate(spec_npy_paths, 1)},
    }
    return row



ファイルをアップロードしてください (WAV形式)...


Saving 渡邊左負荷後⑤.wav to 渡邊左負荷後⑤.wav

解析開始: 渡邊左負荷後⑤.wav
------------------------------
解析完了！ 出力フォルダ: /content/渡邊左負荷後⑤_output_500Hz
  - CSV: 渡邊左負荷後⑤_features.csv
  - 画像: spec_png/ フォルダに保存されました
    (各データにつき '_axis.png' 付きの確認用画像も生成されています)

主要な特徴量:
|   mean_freq |   rms_20_500 |   env_auc |
|------------:|-------------:|----------:|
|     91.6216 |    0.0246339 | 0.0422337 |


## 解析の際は以下のuploadを繰り返す

In [ ]:
# ==========================================
# Google Colab実行用メイン関数
# ==========================================
def main():
    print("ファイルをアップロードしてください (WAV形式)...")
    uploaded = files.upload()

    if not uploaded:
        print("ファイルがアップロードされませんでした。")
        return

    # アップロードされた全ファイルを順次処理
    for filename in uploaded.keys():
        wav_path = Path(filename)
        file_stem = wav_path.stem
        print(f"\n解析開始: {filename}")

        outdir = Path(f"/content/{file_stem}_output_500Hz")
        condition = file_stem
        trial_id = 1

        try:
            row = process_one(wav_path, condition, trial_id, outdir)

            df = pd.DataFrame([row])
            out_csv = outdir / f"{file_stem}_features.csv"
            df.to_csv(out_csv, index=False, encoding="utf-8-sig")

            print("-" * 30)
            print(f"解析完了！ 出力フォルダ: {outdir}")
            print(f"  - CSV: {out_csv.name}")
            print(f"  - 画像: spec_png/ フォルダに保存されました")
            print("    (各データにつき '_axis.png' 付きの確認用画像も生成されています)")

            # 結果の一部を表示
            print("\n主要な特徴量:")
            print(df[['mean_freq', f"rms_{int(BP_LOW)}_{int(BP_HIGH)}", 'env_auc']].to_markdown(index=False))

        except Exception as e:
            print(f"エラーが発生しました ({filename}): {e}")
            import traceback
            traceback.print_exc()

if __name__ == "__main__":
    main()